In [ ]:
import holoviews as hv
hv.extension('bokeh')

In [ ]:
import numpy as np
import scipy.signal

# Transformada de Fourier para señales digitales

## Transformada de Fourier Discreta (DFT)


En este curso nos enfocaremos en el procesamiento de señales usando software

Por ende asumiremos que existe un proceso que sensa y convierte la señal analógica continua en una señal digital, como muestre el siguiente diagrama

<img src="../images/signal-sampling1.png" width="500">

- La señal análogica $s(t)$ se muestrea cada $T_s$ durante un tiempo $T$
- El tiempo asociado a $s_n$ es $t_n = n \cdot T_s$ [s]
- El inverso del periódo de muestreo se denomina **frecuencia de muestreo**: $F_s = \frac{1}{T_s}$ [Hz]
- La cantidad de muestras del arreglo $\{s_n\}$ es equivalente a $N = \lfloor T F_s \rfloor $


Más formalmente diremos que existe un sistema muestreador con frecuencia de muestreo $F_s$ [Hz] tal que

$$
s(t) = \sum_{n=0}^{N-1} s[n] \delta(t - n/F_s),
$$

La transformada de Fourier de esta expresión es

$$
\begin{align}
S(\omega) &= \int s(t) e^{-j\omega t} dt \nonumber \\
&= \int \sum_{n=0}^{N-1} s[n] \delta(t - n/F_s) e^{-j\omega t} dt \nonumber \\
&=  \sum_{n=0}^{N-1} s[n] \int \delta(t - n/F_s) e^{-j\omega t} dt \nonumber \\
&=  \sum_{n=0}^{N-1} s[n] e^{-j\omega n/F_s} \nonumber 
\end{align}
$$

Finalmente definiendo $\omega = 2 \pi f = 2 \pi k \Delta f$ donde $\Delta f = \frac{1}{T} = \frac{F_s}{N}$ y reemplazando obtenemos

$$
S[k] =  \sum_{n=0}^{N-1} s[n] e^{-j \frac{2 \pi}{N} k n},
$$

donde $k = [0, 1, \ldots N-1]$, que se conoce como la **transformada de Fourier discreta** o *discrete Fourier Transform (DFT)

En esta expresión:

- El índice $n$ representa la discretización en el tiempo
- El índice $k$ representa la discretización en frecuencia

### Propiedades de la DFT

La DFT comparte las propiedades de la FT que vimos en la lección anterior

Sin embargo, a diferencia de la FT para señales continuas, la DFT es periódica y su período $N$, como muestra la siguiente expresión

$$
\begin{align}
S[k+N] &= \sum_{n=0}^{N-1} s[n] e^{-j \frac{2 \pi}{N} (k+N) n} \nonumber \\
&=   \sum_{n=0}^{N-1} s[n] e^{-j \frac{2 \pi}{N} k n} e^{-j 2 \pi n} \nonumber \\
&= \sum_{n=0}^{N-1} s[n] e^{-j \frac{2 \pi}{N} k n} = S[k]\nonumber 
\end{align}
$$

### Rango frecuencial de la DFT y frecuencia de Nyquist

Considerando la periodicidad de la DFT, la correspondencia entre el índice entero $k$ y la frecuencia en Hertz es

$$
\begin{matrix}
k &  : & 0, & 1, & 2, & \ldots & N/2 -1, & N/2, & N/2+1, & \ldots & N-2, & N-1 \\
f = k \frac{F_s}{N} & : & 0, & \frac{F_s}{N}, & \frac{2 F_s}{N}, & \ldots & \frac{F_s}{2} - \frac{F_s}{N}, & \frac{F_s}{2} = -\frac{F_s}{2}, &  \frac{F_s}{2} + \frac{F_s}{N} = - \frac{F_s}{2}  + \frac{F_s}{N}, & \ldots & -\frac{2 F_s}{N},  & -\frac{F_s}{N}
\end{matrix}
$$

donde $\frac{F_s}{2}$ se conoce como **frecuencia de Nyquist**, y es la frecuencia más alta a la que se puede calcular la DFT. Hablaremos más sobre esto en la próximo lección

### Interpretación de la DFT es  un producto matricial

Sea $\{s_n\}_{n=0,\ldots,N-1}$ y definiendo 

$$
W_N = e^{-j \frac{2\pi}{N}} = \cos \left(\frac{2\pi}{N}\right) - j \sin \left(\frac{2\pi}{N}\right)
$$

podemos expresar la transformada de Fourier discreta como

$$
S[k] =  \sum_{n=0}^{N-1} s[n] W_N^{kn}, \quad k = [0, 1, \ldots N-1],
$$

que también puede ser expresado matricialmente como

$$
\begin{align}
\begin{pmatrix} 
S[0] \\
S[1] \\
S[2] \\
\vdots \\
S[N-1] \\
\end{pmatrix} &=
\begin{pmatrix}
1 & 1 & 1 & \cdots & 1 \\
1 & W_N & W_N^2 & \cdots & W_N^{N-1} \\
1 & W_N^2 & W_N^4 & \cdots & W_N^{N-2} \\
\vdots & \dots & \dots & \ddots &  \vdots \\
1 & W_N^{N-1} & W_N^{N-2} & \cdots & W_N \\
\end{pmatrix} 
\begin{pmatrix} 
s[0] \\
s[1] \\
s[2] \\
\vdots \\
s[N-1] \\
\end{pmatrix} \nonumber  \\
S &= \Omega s,
\end{align}
$$


Notemos que:

- Por definición $W_N^{kn} = \left(e^{-j \frac{2\pi}{N}}\right)^{kn} = e^{-j \frac{2\pi}{N}kn}$
- Por periodicidad $W_N^{2(N-1)} = W_N^{2(N-1) - N}  = W_N^{N-2}$
- También se tiene simetría hermítica: $W_N^{k(-n)} = W_N^{-kn} = (W_N^{kn})^*$
- $\Omega$ es una matriz cuadrada y simétrica 
- El cálculo de la DFT tiene complejidad cuadrática: $N^2$ multiplicaciones y $N$ sumas


### DFT inversa

Sea $S=\{S[0], S[1], S[2], S[3]\}$, la DFT de una señal $s=\{s[0], s[1], s[2], s[3]\}$. 

Como vimos anteriormente podemos escribir la relación entre $S$ y $s$ como el siguiente sistema de ecuaciones

$$
S= 
\begin{pmatrix} 
S[0] \\
S[1] \\
S[2] \\
S[3] 
\end{pmatrix} =
\begin{pmatrix}
1 & 1  & 1 & 1\\
1 & W_4  & W_4^2 & W_4^3 \\
1 & W_4^2  & W_4^4 & W_4^2 \\
1 & W_4^3  & W_4^2 & W_4 \\
\end{pmatrix} 
\begin{pmatrix} 
s[0] \\
s[1] \\
s[2] \\
s[3] 
\end{pmatrix} = \begin{pmatrix}
1 & 1  & 1 & 1\\
1 & -j  & -1 & j \\
1 & -1  & 1 & -1 \\
1 & j  & -1 & -j \\
\end{pmatrix} 
\begin{pmatrix} 
s[0] \\
s[1] \\
s[2] \\
s[3] 
\end{pmatrix} = \Omega s
$$

Si conocemos $S$ y queremos encontrar $s$, sólo necesitamos invertir la matriz $\Omega$

Por ejemplo

In [ ]:
np.set_printoptions(precision=4, suppress=True)

N = 4
WN = np.exp(-1j*2.0*np.pi/N)
index = np.array(range(N))[:, np.newaxis]
Omega = WN**(index*index.T)

Omega

In [ ]:
# La siguiente función calcula el inverso de una matriz cuadrada
np.linalg.inv(Omega) 

Como veremos a continuación la matriz $\Omega$ de la DFT es una matriz bastante especial. 

El inverso de $\Omega$ es equivalente a su complejo conjugado dividido N. El complejo conjugado de una matriz es una matriz donde todos los elementos complejos se reemplazan por su conjugado 

$$
(a + j b)^{*} = a - j b
$$

En este caso particular ($N=4$) tenemos que el inverso de la matriz es 

$$
\Omega^{-1} = \frac{1}{4}
\begin{pmatrix}
1 & 1 & 1 &  1 \\
1 & j & -1 &  -j \\
1 & -1 & 1 &  -1 \\
1 & -j & -1  & j \\
\end{pmatrix} = \frac{1}{4} \Omega^*
$$

Notar que esta propiedad se cumple para todo $N$. Esto es muy útil pues el costo de obtener el complejo conjugado es mucho menor que el de invertir una matriz cuadrada arbitraria

Resumiendo, podemos recuperar $s$ a partir de $S$ usando

$$
s = \frac{1}{N} \Omega^* S
$$

o lo que es equivalente

$$
s[n] = \frac{1}{N} \sum_{k=0}^{N-1} S[k] W_N^{-kn}, \quad n = [0, 1, \ldots N-1]
$$

que se conoce como la DFT inversa y cuya únicas diferencias con la DFT son el factor $\frac{1}{N}$ y el signo del exponente

## Transformada Rápida de Fourier (FFT)

Como vimos antes la computación de la DFT tiene complejidad $\mathcal{O}(N^2)$, lo cual es muy costoso de aplicar en la práctica

Existe una aproximación exacta de la DFT con complejidad $\mathcal{O}(N\log N)$: la **Fast Fourier Transform** (FFT). 

En esta lección veremos el algoritmo de Cooley-Tukey, un algoritmo FFT, que se basa en expresiones recursiva que explotan las simetrías que vimos en la matriz $\Omega$

La DFT se separa en dos "medias" DFT, donde una mitad se enfoca en los índices pares (even) y la otra en los impares (odd) como muestra la siguiente ecuación

$$
\begin{align}
S[k] &=  \sum_{n=0}^{N-1} s[n] W_N^{kn} \nonumber \\
&= \sum_{n=0}^{N/2-1} s[2n] W_N^{k 2n} + \sum_{n=0}^{N/2-1} s[2n+1] W_N^{k(2n+1)} \nonumber \\
&= \sum_{n=0}^{N/2-1} s[2n] W_{N/2}^{kn} + W_N^{k} \sum_{n=0}^{N/2-1} s[2n+1] W_{N/2}^{kn} \nonumber \\
&= S_E[k] + W_N^{k} S_O[k] ~~ \forall k \in [0,N/2]  \nonumber 
\end{align} 
$$

Luego por periodicidad de la DFT tenemos que

$$
\begin{align}
S_E[k + N/2] &=  \sum_{n=0}^{N/2-1} s[2n] W_{N/2}^{(k+N/2)n} \nonumber \\
&=  \sum_{n=0}^{N/2-1} s[2n] W_{N/2}^{kn} \exp \left(-j2\pi n \right) = S_E[k], \nonumber
\end{align}
$$

e igualmente

$$
S_O[k + N/2] = S_O[k],
$$

juntando ambos tenemos que

$$
\begin{align}
S[k + N/2] &=  S_E[k + N/2] + W_{N}^{(k+N/2)} S_O[k + N/2] \nonumber  \\
&=  S_E[k] + W_{N}^{k} \exp \left(-j\pi\right) S_O[k] \nonumber \\
&=  S_E[k] - W_{N}^{k} S_O[k] \nonumber 
\end{align}
$$

es decir que

$$
\begin{align}
S[k] &=  S_E[k] + W_{N}^{k} S_O[k] \nonumber \\
S[k + N/2] &=  S_E[k] - W_{N}^{k} S_O[k] \quad \forall k \in [0,N/2]  \nonumber 
\end{align}
$$

La DFT de $k$ y $k+N/2$ difieren sólo en un signo. Cada ves que dividimos el intervalo podemos reducir los cómputos a la mitad. Podemos repetir la división $\log N$ veces por lo que el costo de la FFT es $N \log N$

La siguiente esquema muestra la FFT para una señal $x$ de largo $N=16$, en $\log_2 16=4$  pasos se calculan todos los componentes de su espectro $X$

<img src="../images/fft-16samples.png">

Imagen tomada de: [http://www.themobilestudio.net/the-fourier-transform-part-14](http://www.themobilestudio.net/the-fourier-transform-part-14)

La siguiente tabla compara la cantidad de multiplicaciones entre la DFT y la FFT

| N | DFT | FFT | FFT/DFT [%] |
|---|---|---|---|
| 32 | 1024 | 160 | 15.6 |
| 128 | 16,384 | 896 | 5.46 |
| 1,024 | 1,048,576 | 10,240 | 0.97 |


## Implementación de la FFT en Python

Existen implementaciones de la FFT en múltiples lenguajes de programación. Entre las más famosas y eficientes destacan la librería [`FFTPACK`](https://www.netlib.org/fftpack/) para lenguaje Fortran y la librería [`FFTW`](https://www.fftw.org/) para lenguaje C. Python cuenta con varios *wrappers* a estas librerías, por ejemplo   [`pyFFTW` ](https://hgomersall.github.io/pyFFTW/) permite usar las funciones de `FFTW` desde Python

En este curso utilizaremos principalmente la sub-librería [`scipy.fft`](https://docs.scipy.org/doc/scipy/reference/fft.html) disponible desde la versión 1.4 de la librería de cómputo científico `scipy` para lenguaje Python. Previo a dicha versión `scipy` contaba con un *wrapper* de la librería `FFTPACK` en `scipy.fftpack` que aun está disponible por retrocompatibilidad. 

> La recomendación general es usar `scipy.fft` que es una reimplementación moderna de los algoritmos de `FFTPACK` con mejoras importantes en desempeño como por ejemplo la posibilidad de ejecutar sus rutinas en paralelo. 

A continuación veremos algunas de las funciones que ofrece en detalle

### Función `scipy.fft.fft` y variantes

Para calcular la FFT de un arreglo de números complejos o reales se usa

```python
scipy.fft.fft(x: np.ndarray, # Un arreglo de NumPy,
              n: int=None, # El número de muestras que se usarán para calcular la FFT
              axis: int=-1, # El eje del arreglo en que se calculara la FFT,
              workers: Optional[int]=None, # Número de trabajadores para calcular la FFT en paralelo
              norm: {"backward", "ortho", "forward"}, # El tipo de normalización
              ...
             )
```

Cabe destacar que:

- Si no se especifica el argumento `n` entonces `n=len(x)`. 
    - Por otro lado si especificamos `n` y `n<len(x)` entonces `x` se truncara para quedar de largo `n`. 
    - En cambio si `n>len(x)` entonces se agregarán ceros al final de `x` para alargarlo (padding) 
- Si `x` es un arreglo multidimensional entonces la FFT se calculará sólo en la dimensión especificada por el argumento `axis` (por defecto la última). 

:::{note} 

Si trabajamos con señales unidimensionales podemos ignorar dicho argumento. En cambio si estamos trabajando con señales bidimensionales y multidimensionales la recomendación es usar las funciones `scipy.fft.fft2` y `scipy.fft.fftn`, respectivamente

:::

Otro detalle es que `scipy.fft.fft` funciona tanto para números reales como complejos, sin embargo si la señal `x` es de números reales entonces es más conveniente usar la función `scipy.fft.rfft` para evitar redundancia en el espectro. Más adelante explicaremos esto en más detalle pero primero pasemos a un ejemplo


**Ejemplo** Sea una señal con 

- una exponencial compleja con frecuencia $1$ Hz, amplitud $1$ y desfase $0$ grados
- una exponencial compleja con frecuencia $-4$ Hz, amplitud $1/4$ y desfase  $\pi/3 \approx 1.0472$

:::{note}

La [frecuencia negativa](https://en.wikipedia.org/wiki/Negative_frequency) como concepto sólo tiene sentido en una señal compleja. Puedes imaginarlo como el sentido en que rota la señal en el plano complejo (horario o antihorario)

:::


La señal está muestreada a 40 Hz

In [ ]:
Fs = 40
t = np.arange(0, 5, step=1./Fs); 
s = np.exp(1j*(2.0*np.pi*t)) + 0.25*np.exp(1j*(2.0*np.pi*-4*t + np.pi/3))
display(s.shape, s.dtype)

Para graficar la señal debemos graficar la parte real e imaginaria por separado

In [ ]:
real_plot = hv.Curve((t, np.real(s)), 'Tiempo', 'Señal', label='Real')
imag_plot = hv.Curve((t, np.imag(s)), 'Tiempo', 'Señal', label='Imaginaria')
(real_plot * imag_plot).opts(width=500)

El espectro que retorna la función `scipy.fft.fft` es una secuencia de números complejos

In [ ]:
S = scipy.fft.fft(s, norm='forward')
display(S.dtype, S.shape)

Al igual que con la señal podemos graficar su parte real e imaginaria

In [ ]:
real_plot = hv.Curve((np.real(S)), 'Frecuencia', 'Espectro', label='Real')
imag_plot = hv.Curve((np.imag(S)), 'Frecuencia', 'Espectro', label='Imaginario')
(real_plot + imag_plot).opts(width=500)

Pero en general, el espectro es más fácil de interpretar si lo separaremos en valor absoluto y ángulo (coordenadas polares)

In [ ]:
SA = np.absolute(S) # Espectro de magnitud 
SP = np.angle(S) # Espectro fase
display(SA.dtype, SP.dtype)

In [ ]:
abs_plot = hv.Curve((np.abs(S)), 'Frecuencia', 'Espectro de magnitud')
ang_plot = hv.Curve((np.angle(S)), 'Frecuencia', 'Espectro de fase')
(abs_plot + ang_plot).opts(width=500)

Hasta ahora hemos omitido definir el eje de las frecuencias

Si leemos la documentación veremos que `scipy.fft.fft` retorna un arreglo de Numpy de números complejos que corresponden al espectro y que por convención están ordenados en frecuencia según 

$$
f = \left[0, \frac{F_s}{N},  \frac{2 F_s}{N}, \ldots, \frac{F_s}{2},  \ldots  -\frac{2 F_s}{N},   -\frac{F_s}{N} \right]
$$ 

es decir que primero van las frecuencias positivas y luego las negativas en orden invertido

### Función utilitarias `scipy.fft.fftfreq` y `scipy.fft.fftshift`

Para graficar el espectro resulta muy útil la función

```python
scipy.fft.fftfreq(n, #  El número de muestras de x
                  d=1.0 # El inverso de la frecuencia de muestreo
                 )
```

Que retorna el arreglo de frecuencias mostrado anteriormente. Existe también la función `scipy.fft.rfftfreq` que retorna las frecuencias asociadas a `scipy.fft.rfft`

Otra función que facilita la lectura del espectro es 

```python
scipy.fft.fftshift(S, #  Un arreglo de numpy
                  ...
                  )
```

que sirve para dar vuelta la primera y segunda mitad de `S` obtiendo un orden en frecuencia de menor a mayor como es más natural. 

Veamos como se ve el espectro usando estas funciones

In [ ]:
# Se crean las frecuencias y se les hace el shift
freqs = scipy.fft.fftshift(scipy.fft.fftfreq(n=len(s), d=1./Fs))
# Luego le hago el shift al espectro
S = scipy.fft.fftshift(S)

In [ ]:
abs_plot = hv.Curve((freqs, np.abs(S)), 'Frecuencia [Hz]', 'Espectro de magnitud')
ang_plot = hv.Curve((freqs, np.angle(S)), 'Frecuencia [Hz]', 'Espectro de fase')
(abs_plot + ang_plot).opts(width=500)

Del espectro de magnitud podemos ver que practicamente todas las frecuencias tienen magnitud nula a excepción de dos *peaks*

El espectro de fase (ángulo) parece un poco más ruidoso, pero debemos recordar que sólo interesan los ángulos de los componentes de magnitud no nula

¿Cuáles son estas frecuencias importantes?

In [ ]:
# Ordenamos las magnitudes de mayor a menor y obtenemos su índice
idx = np.argsort(np.abs(S))[::-1]
# Finalmente observamos las frecuncies, magnitudes y ángulos de los dos principales
for k, i in enumerate(idx[:2]):
    print(f"({k}), Frecuencia: {freqs[i]:0.2f} Hz\t Magnitud: {np.abs(S)[i]:0.4f}\t Angulo: {np.angle(S)[i]:0.4f}")

:::{note}

Cada peak corresponde a uno de los componentes de la señal

:::

Esto es esperable pues la transformada de Fourier descompone las señales en exponenciales complejas



### Ejemplos de espectros

El sitio web: https://www.thefouriertransform.com/pairs/fourier.php tiene un listado de distintas señales y sus transformadas de Fourier analíticas. 

A continuación se muestran los espectros obtenidos con la FFT de algunas de estas señales. Contraste lo que observa con el resultado teórico

In [ ]:
# Señal coseno
Fs = 40
t = np.arange(0, 5, step=1./Fs)
s = np.cos(2.0*np.pi*t*1)
f = scipy.fft.fftshift(scipy.fft.fftfreq(n=len(s), d=1/Fs))
S = scipy.fft.fftshift(scipy.fft.fft(s))

real_plot = hv.Curve((f, np.real(S)), 'Frecuencia', 'Espectro', label='Real')
imag_plot = hv.Curve((f, np.imag(S)), 'Frecuencia', 'Espectro', label='Imaginario')
(real_plot + imag_plot).opts(width=500)

In [ ]:
# Señal seno
Fs = 40
t = np.arange(0, 5, step=1./Fs)
s = np.sin(2.0*np.pi*t*1)
f = scipy.fft.fftshift(scipy.fft.fftfreq(n=len(s), d=1/Fs))
S = scipy.fft.fftshift(scipy.fft.fft(s))

real_plot = hv.Curve((f, np.real(S)), 'Frecuencia', 'Espectro', label='Real')
imag_plot = hv.Curve((f, np.imag(S)), 'Frecuencia', 'Espectro', label='Imaginario')
(real_plot + imag_plot).opts(width=500)

In [ ]:
# Señal Gaussiana
Fs = 40
t = np.arange(0, 5, step=1./Fs)
s = np.exp(-10*t**2)
f = scipy.fft.fftshift(scipy.fft.fftfreq(n=len(s), d=1/Fs))
S = scipy.fft.fftshift(scipy.fft.fft(s))

real_plot = hv.Curve((f, np.real(S)), 'Frecuencia', 'Espectro', label='Real')
imag_plot = hv.Curve((f, np.imag(S)), 'Frecuencia', 'Espectro', label='Imaginario')
(real_plot + imag_plot).opts(width=500)

In [ ]:
# Señal rectangular
Fs = 40
t = np.arange(0, 5, step=1./Fs)
s = np.zeros_like(t)
s[np.abs(t)<1] = 1
f = scipy.fft.fftshift(scipy.fft.fftfreq(n=len(s), d=1/Fs))
S = scipy.fft.fftshift(scipy.fft.fft(s))

real_plot = hv.Curve((f, np.real(S)), 'Frecuencia', 'Espectro', label='Real')
imag_plot = hv.Curve((f, np.imag(S)), 'Frecuencia', 'Espectro', label='Imaginario')
(real_plot + imag_plot).opts(width=500)

### Diferencia en desempeño 

Comparemos ahora la diferencia en tiempo de cómputo entra la DFT calculada como producto matricial y la FFT 

Usaremos la magia `timeit` de IPython para medir el tiempo de las rutinas

In [ ]:
time_dft, time_fft = [], []
for N in [2, 5, 10, 20, 50, 100, 200, 1000]:
    t = np.linspace(0, N-1, num=N)
    s = np.cos(2.0*np.pi*t)
    ans=%timeit -q -o -n10 -r10 scipy.fft.fftshift(scipy.fft.fft(s))
    time_fft.append(ans)
    f = scipy.fft.fftfreq(n=N, d=1)
    tf = t[:, np.newaxis]*f[:, np.newaxis].T
    ans=%timeit -q -o -n10 -r10 np.dot(s, np.exp(-1j*2.0*np.pi*tf))
    time_dft.append(ans)
    #display(np.allclose(scipy.fft.fft(s), np.dot(s, np.exp(-1j*2.0*np.pi*tf))))

In [ ]:
plot_time_dft = hv.Curve((np.array([2, 5, 10, 20, 50, 100, 200, 1000]), 
                          np.array([time.average for time in time_dft])), 'N', 'Tiempo [s]', label='DFT')
plot_time_fft = hv.Curve((np.array([2, 5, 10, 20, 50, 100, 200, 1000]), 
                          np.array([time.average for time in time_fft])), 'N', 'Tiempo [s]', label='FFT')
(plot_time_dft * plot_time_fft).opts(width=500, logx=True, logy=True, legend_position='top_left')

- Para $N<20$ la DFT es más rápida, aunque la diferencia no es muy notoria. 
- Para $N>20$ la FFT mantiene un tiempo casi constante, mientras que la DFT termina siendo casi dos órdenes de magnitud más costosa